![GenAI Banner](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)


<p><font size="5" color='grey'> <b>
Agenten
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/Python_Modules
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN', 'SERPAPI_API_KEY', 'TAVILY_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()
# Bei Bedarf: Trennen zwischen Installationsname () und Importname (für Python) beide Angaben in Klammern
# install_packages([('markitdown[all]', 'markitdown'), 'langchain_chroma', ]

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([('google-search-results', 'serpapi'), ('tavily_python', 'tavily'),  'wikipedia'])

# 1 | Was ist ein echter Agent?
---

Wenn man sich mit generativer KI beschäftigt, stößt man früher oder später auf den Begriff Agent – also ein System, das Aufgaben eigenständig ausführt. Doch was genau ist ein *„echter“* Agent? Muss er vollständig autonom sein? Solche Fragen führen schnell zu endlosen Grundsatzdiskussionen – und genau das ist nicht hilfreich, wenn man einfach anfangen möchte, mit GenAI zu arbeiten.

Der bekannte KI-Experte **Andrew Ng** hat einen anderen, praktischeren Vorschlag gemacht: Statt darüber zu streiten, ob etwas ein Agent ist oder nicht, sollten wir lieber davon sprechen, wie **agentisch** ein System ist – also wie **selbstständig** es arbeitet. So kann man sich auf das konzentrieren, was wirklich zählt: Was kann das System leisten, und wo kann es sinnvoll eingesetzt werden?

Besonders einfach ist der Einstieg bei eher einfachen Aufgaben – also bei Prozessen, die heute noch manuell erledigt werden, wie das Ausfüllen von Formularen, das Nachschlagen in einer Datenbank oder das Kopieren von Informationen zwischen verschiedenen Systemen. Diese Aufgaben lassen sich gut in sogenannte agentische Workflows überführen – also in Abläufe, bei denen die KI (teilweise) selbstständig handelt.

Natürlich gibt es auch deutlich komplexere Anwendungen, bei denen die KI viele Entscheidungen trifft, Schleifen durchläuft und sich an neue Situationen anpasst. Solche Systeme sind spannend – aber gerade für den Anfang ist es oft sinnvoller, mit kleineren, überschaubaren Schritten zu starten. Dort liegen aktuell auch die meisten Chancen, GenAI im Alltag oder im Beruf sinnvoll einzusetzen.

**Fazit:**     
Es muss nicht gleich ein „superintelligenter“ Agent sein. Besser ist es, pragmatisch zu denken, einfache Prozesse zu automatisieren – und so Schritt für Schritt Erfahrungen zu sammeln.

<img src="https://substackcdn.com/image/fetch/w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F9e571420-ef02-421c-a8f6-e79fdfe64973_988x1122.gif" width="600" alt="Avatar">


[Quelle](https://substackcdn.com/image/fetch/w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F9e571420-ef02-421c-a8f6-e79fdfe64973_988x1122.gif)



<br>

**Agenten erweitern LLMs**

Agenten sind mehr als nur Sprachmodelle. Sie verbinden die Sprachfähigkeiten eines LLMs mit praktischer Handlungsfähigkeit:

+ LLM als Denkmodul: Das LLM übernimmt das Sprachverständnis und das logische Schlussfolgern.

+ Erweiterbarkeit durch Tools: Agenten greifen z. B. auf Web, Datenbanken oder externe APIs zu.

+ Transparenter Denkprozess: Der Agent zeigt, wie er zum Ergebnis kommt – ideal zum Lernen und Verstehen.

+ Entscheidungen treffen: Agenten analysieren, planen und wählen aus mehreren Optionen – nicht nur einmal, sondern iterativ.

# 2 | Direkter Vergleich
---

## 2.1 Setup und Tools

Bevor wir vergleichen können, müssen wir die notwendigen Tools für unseren Agenten definieren. Diese Tools repräsentieren die erweiterten Fähigkeiten, die einem einfachen LLM fehlen.



In [ ]:
# Einfache Tools definieren
from langchain_core.tools import Tool
from langchain_community.utilities.serpapi import SerpAPIWrapper
import requests

# Tool 1
def simple_calculator(expression):
    """Einfacher Rechner - Das kann ein LLM oft nicht präzise"""
    try:
        # Sicherheitscheck
        if any(x in expression for x in ['import', 'exec', '__']):
            return "Unsichere Operation"
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "Berechnungsfehler"

# Tool 2
def get_weather(city):
    """Aktuelle Wetterdaten - Das kann ein LLM GAR NICHT"""
    # Vereinfachte Demo-Version
    return f"🌤️ Aktuelles Wetter in {city}: 22°C, sonnig (Demo-Version)"

# Tool 3
serpapi = SerpAPIWrapper()

# Tool-Liste erstellen
tools = [
    Tool(
        name="internet_search",
        func=serpapi.run,
        description="🌐 INTERNETSUCHE - Aktuelle Informationen finden (was LLM NICHT kann)"
    ),
    Tool(
        name="calculator",
        func=simple_calculator,
        description="🔢 RECHNER - Präzise Berechnungen (was LLM oft falsch macht)"
    ),
    Tool(
        name="weather",
        func=get_weather,
        description="🌤️ WETTER - Echtzeitdaten abrufen (was LLM unmöglich ist)"
    )
]

print("✅ Tools definiert:")
for tool in tools:
    print(f"   • {tool.name}: {tool.description}")

## 2.2 Vergleichstest

Wir verwenden eine Frage, die sowohl aktuelle Daten als auch eine Berechnung erfordert, um die Grenzen eines LLMs und die Stärken eines Agenten zu demonstrieren.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import time

# LLM Setup
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

# Test-Frage die Grenzen aufzeigt
test_question = "Wie ist das Wetter in Berlin und was ist 2847 * 1923?"

mprint("### 🧪 VERGLEICHSTEST")
mprint("---")
mprint(f"**Frage:** {test_question}")
print()

# 1. EINFACHES LLM PROBIEREN
mprint("### 1️⃣ EINFACHES LLM:")
mprint("---")

start_time = time.time()
llm_response = llm.invoke(test_question)
llm_time = time.time() - start_time

mprint(f"**Antwort:** {llm_response.content}")
mprint(f"**Zeit:** {llm_time:.2f}s")
print()
mprint("### ❌ PROBLEME:")
mprint("+ Keine aktuellen Wetterdaten")
mprint("+ Berechnung möglicherweise ungenau")
mprint("+ Kann nicht überprüfen ob Antwort stimmt")
print()

# 2. AGENT MIT TOOLS
mprint("### 2️⃣ AGENT MIT TOOLS:")
mprint("---")

# Agent erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Agent. Nutze Tools für aktuelle Daten und Berechnungen."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Zeigt Denkprozess
    return_intermediate_steps=True
)

start_time = time.time()
agent_response = agent_executor.invoke({"input": test_question})
agent_time = time.time() - start_time

mprint(f"**Antwort:**  {agent_response['output']}")
mprint(f"**Zeit:** {agent_time:.2f}s")
print()
mprint("### ✅ VORTEILE:")
mprint("+ Aktuelle Wetterdaten abgerufen")
mprint("+ Präzise Berechnung durchgeführt")
mprint("+ Transparenter Denkprozess sichtbar")
mprint("+ Schritte nachvollziehbar")


## 2.3 Unterschiede



Die Unterschiede zwischen LLM und Agent lassen sich in fünf Kernbereichen zusammenfassen, die den Paradigmenwechsel von statischer zu dynamischer KI verdeutlichen.

**Vergleich der Fähigkeiten:**

| Aspekt | Einfaches LLM | Agent |
|--------|---------------|-------|
| **Aktuelle Daten** | ❌ Nur Trainingsdaten | ✅ Über Tools |
| **Berechnungen** | ⚠️ Oft ungenau | ✅ Präzise Tools |
| **Externe APIs** | ❌ Unmöglich | ✅ Beliebig erweiterbar |
| **Transparenz** | 🔒 Verborgen | 👁️ Sichtbar |
| **Erweiterbarkeit** | ❌ Statisch | ✅ Modular |



**🎯 FAZIT:**
- **Agent = LLM + Tools + Reasoning**
- ➡️ Aus reaktiv wird proaktiv
- ➡️ Aus statisch wird dynamisch
- ➡️ Aus isoliert wird vernetzt


# 3 | Anatomie eines Agenten
---



Um Agenten effektiv einsetzen zu können, muss man  ihre innere Struktur verstehen. Ein Agent besteht aus vier Hauptkomponenten, die zusammenarbeiten, um komplexe Aufgaben zu lösen. Diese Architektur ermöglicht es, die Stärken von LLMs mit praktischen Werkzeugen zu kombinieren.

**Die 4 Kern-Komponenten**

Jede Komponente eines Agenten hat eine spezifische Rolle im Gesamtsystem. Das Verständnis dieser Rollen hilft beim Design eigener Agenten und bei der Fehlersuche.

+ **LLM (Das Gehirn)**: Das Large Language Model fungiert als zentrale Intelligenz des Agenten. Es versteht die Benutzeranfrage, interpretiert Tool-Ergebnisse und entscheidet über nächste Schritte. Ohne das LLM wäre der Agent nur eine Sammlung unverbundener Werkzeuge.

+ **Tools (Die Hände)**: Tools sind spezialisierte Funktionen, die dem Agenten erlauben, mit der Außenwelt zu interagieren. Sie können so einfach sein wie ein Rechner oder so komplex wie eine Datenbankverbindung. Jedes Tool erweitert die Fähigkeiten des Agenten erheblich.

+ **Agent-Scratchpad (Das Gedächtnis)**: Das Scratchpad speichert Zwischenergebnisse und den Verlauf der Tool-Aufrufe. Es ermöglicht dem Agenten, aus vorherigen Schritten zu lernen und komplexe, mehrstufige Reasoning-Prozesse durchzuführen.

+ **Executor (Der Koordinator)**: Der Executor orchestriert das Zusammenspiel aller Komponenten. Er entscheidet, wann Tools aufgerufen werden, wann genug Informationen gesammelt wurden und wann die finale Antwort generiert werden soll.


<br>

**Der Agent-Denkprozess**

Der Denkprozess eines Agenten ist fundamental anders als der eines LLMs. Während ein LLM linear von Eingabe zu Ausgabe arbeitet, durchläuft ein Agent einen iterativen Zyklus aus Reasoning, Tool-Nutzung und Bewertung.

+ **Schritt 1 - Verstehen**: Der Agent analysiert die Benutzeranfrage und identifiziert, welche Informationen oder Aktionen benötigt werden. Dies geschieht durch das LLM, das die natürliche Sprache interpretiert.

+ **Schritt 2 - Planen**: Basierend auf dem Verständnis der Anfrage plant der Agent, welche Tools in welcher Reihenfolge verwendet werden sollten. Diese Planung kann sich während der Ausführung ändern.

+ **Schritt 3 - Ausführen**: Der Agent ruft das gewählte Tool auf und erhält ein Ergebnis. Dieses Ergebnis wird im Scratchpad gespeichert und steht für weitere Entscheidungen zur Verfügung.

+ **Schritt 4 - Bewerten**: Nach jedem Tool-Aufruf bewertet der Agent, ob genügend Informationen vorliegen oder weitere Schritte notwendig sind. Diese Bewertung bestimmt den weiteren Verlauf.

+ **Schritt 5 - Iterieren oder Antworten**: Je nach Bewertung kehrt der Agent zu Schritt 2 zurück oder generiert die finale Antwort. Diese Flexibilität ermöglicht die Lösung komplexer, unvorhersehbarer Probleme.



# 4 | Praktische Beispiele
---

Praktische Beispiele helfen dabei, die Konzepte von Agenten greifbar zu machen. Wir beginnen mit einfachen, fokussierten Agenten und steigern die Komplexität schrittweise. Diese Progression zeigt, wie Agenten für verschiedene Anwendungsfälle optimiert werden können.



## 4.1 Rechner-Agent

Ein Rechner-Agent demonstriert die Grundprinzipien der Agent-Architektur mit minimaler Komplexität. Er zeigt, wie ein Agent eine spezifische Domäne (Mathematik) abdecken kann, ohne von anderen Funktionen abgelenkt zu werden.


**🔢 RECHNER-AGENT DEMO**

**Konfiguration:** Agent nur mit Calculator-Tool ausgestattet

**Test-Fragen und Ergebnisse:**
- **"Was ist 15 * 23?"** → ✅ 345
- **"Berechne (100 + 50) * 3"** → ✅ 450  
- **"Was ist 2 hoch 10?"** → ✅ 1024

**Erkenntnisse:**
- **Fokussierung** auf eine Domäne macht den Agenten zuverlässiger
- **Wiederverwendbarkeit** derselben Tool-Logik für verschiedene Probleme
- **Transparenz** durch sichtbare Reasoning-Schritte


Dieser Agent zeigt drei wichtige Prinzipien: **Fokussierung** auf eine Domäne macht den Agenten zuverlässiger, **Wiederverwendbarkeit** derselben Tool-Logik für verschiedene Probleme, und **Transparenz** durch sichtbare Reasoning-Schritte.

## 4.2 Recherche-Agent



Ein Recherche-Agent erweitert die Fähigkeiten erheblich, indem er Zugang zu aktuellen Informationen erhält. Dies zeigt, wie Agenten die fundamentale Begrenzung von LLMs (veraltete Trainingsdaten) überwinden.


**🔍 RECHERCHE-AGENT DEMO**

**Konfiguration:** Agent nur mit Internet Search-Tool ausgestattet

**Test-Frage:** "Was sind die neuesten Nachrichten über künstliche Intelligenz?"

**Verhalten:**
- Agent ruft automatisch das Internet Search-Tool auf
- Durchsucht aktuelle Nachrichtenquellen
- Fasst relevante Informationen zusammen

**Beispiel-Ergebnis:**
"Aktuelle KI-Entwicklungen umfassen Durchbrüche in der Sprachverarbeitung, neue Robotik-Anwendungen in der Industrie und verstärkte Diskussionen über KI-Regulierung in Europa..."

**Key-Features:**
- **Aktualität** durch Zugriff auf Live-Daten
- **Validierung** von Informationen durch mehrere Quellen
- **Kontextualisierung** von Suchergebnissen für die spezifische Anfrage


Der Recherche-Agent demonstriert **Aktualität** durch Zugriff auf Live-Daten, **Validierung** von Informationen durch mehrere Quellen, und **Kontextualisierung** von Suchergebnissen für die spezifische Anfrage.



## 4.3 Multi-Tool Agent



Ein Multi-Tool Agent kombiniert verschiedene Fähigkeiten und zeigt die wahre Stärke der Agent-Architektur: die Orchestrierung mehrerer Tools zur Lösung komplexer, mehrdimensionaler Probleme.


**🛠️ MULTI-TOOL AGENT DEMO**

**Konfiguration:** Agent mit allen verfügbaren Tools (Internet, Rechner, Wetter)

**Komplexe Test-Anfrage:**
"Ich plane eine Reise nach München nächste Woche. Kannst du das aktuelle Wetter dort prüfen und berechnen was 3 Hotelübernachtungen à 120€ plus 2 Zugtickets à 89€ kosten würden?"

**Agent-Verhalten:**
1. **Wetter-Tool** → Aktuelle Wetterdaten für München abrufen
2. **Calculator-Tool** → Reisekosten berechnen: (3 × 120€) + (2 × 89€) = 538€
3. **Synthese** → Beide Informationen in kohärente Antwort zusammenfassen

**Beispiel-Antwort:**
"Das aktuelle Wetter in München beträgt 18°C mit teilweise bewölktem Himmel - ideal für eine Städtereise. Die Gesamtkosten für Ihre Reise belaufen sich auf 538€ (360€ für Hotels + 178€ für Zugtickets)."

**Besondere Fähigkeiten:**
- **Tool-Orchestrierung** durch intelligente Auswahl und Sequenzierung
- **Kontext-Erhaltung** zwischen verschiedenen Tool-Aufrufen
- **Synthesefähigkeit** beim Kombinieren verschiedener Informationstypen


Dieser Agent zeigt **Tool-Orchestrierung** durch intelligente Auswahl und Sequenzierung, **Kontext-Erhaltung** zwischen verschiedenen Tool-Aufrufen, und **Synthesefähigkeit** beim Kombinieren verschiedener Informationstypen.

# 5 | Wann braucht man einen Agent?
---

Die Entscheidung zwischen einem einfachen LLM und einem Agenten hängt von den spezifischen Anforderungen Ihrer Anwendung ab. Eine klare Entscheidungsmatrix hilft dabei, die richtige Technologie für den jeweiligen Anwendungsfall zu wählen und Ressourcen effizient einzusetzen.



Die Wahl der richtigen Technologie beginnt mit der Analyse der Aufgabenanforderungen. Während LLMs für viele Textverarbeitungsaufgaben ausreichen, sind Agenten unverzichtbar, wenn externe Interaktionen oder aktuelle Daten benötigt werden.

**Verwenden Sie einen Agenten wenn:**

+ Sie aktuelle oder dynamische Daten benötigen, die sich häufig ändern (Aktienkurse, Wetter, Nachrichten). Agenten können über APIs auf Live-Daten zugreifen und diese in ihre Antworten integrieren.

+ Präzise Berechnungen erforderlich sind, bei denen Genauigkeit kritisch ist. LLMs approximieren mathematische Operationen, während Agenten echte Rechner-Tools verwenden.

+ Externe Systeme angesprochen werden müssen, wie Datenbanken, APIs oder andere Services. Agenten können diese Integrationen nahtlos abwickeln.

+ Komplexe, mehrstufige Prozesse durchgeführt werden sollen, bei denen jeder Schritt vom vorherigen abhängt. Der Agent-Reasoning-Loop ist für solche Szenarien optimiert.

**Ein einfaches LLM reicht wenn:**

+ Reine Textverarbeitung ohne externe Daten im Fokus steht. Für Zusammenfassungen, Übersetzungen oder Textanalysen sind LLMs optimal.

+ Kreative Aufgaben gelöst werden sollen, wie das Schreiben von Geschichten, Gedichten oder Marketing-Texten. Hier sind die kreativen Fähigkeiten des LLMs gefragt.

+ Erklärungen oder Bildungsinhalt basierend auf allgemeinem Wissen benötigt werden. LLMs haben Zugang zu einem enormen Wissensfundus.

+ Statische Code-Generierung ohne externe Abhängigkeiten erforderlich ist. Für einfache Programmieraufgaben sind LLMs sehr effektiv.

<br>

**Die Faustregel lautet**:    
Wenn Sie Tools, aktuelle Daten oder externe Interaktionen benötigen, wählen Sie einen Agenten. Für reine Textverarbeitung reicht ein LLM aus.

# 6 | Hands-On: Agent bauen
---

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain.utilities import WikipediaAPIWrapper
from tavily import TavilyClient
import os

In [ ]:
# Tools definieren

def read_file(filename):
    try:
        with open(filename, 'r') as f:
            return f.read()
    except:
        return f"Datei {filename} nicht gefunden"

def write_file(content):
    filename, text = content.split("|", 1)
    with open(filename, 'w') as f:
        f.write(text)
    return f"Datei {filename} geschrieben"

# Tavily Web-Suche
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))  # API-Key als Umgebungsvariable setzen

def web_search(query):
    try:
        result = tavily.search(query=query, search_depth="basic")
        return f"Suchergebnis: {result['results'][0]['content']}"
    except Exception as e:
        return f"Fehler bei Tavily-Suche: {e}"

# Wikipedia-Suche
def wiki_search(term):
    wiki = WikipediaAPIWrapper()
    try:
        return wiki.run(term)
    except Exception as e:
        return f"Wikipedia-Fehler: {e}"

custom_tools = [
    Tool(name="read_file", func=read_file, description="Datei lesen"),
    Tool(name="write_file", func=write_file, description="Datei schreiben (Format: datei.txt|inhalt)"),
    Tool(name="search", func=web_search, description="Führe eine Websuche mit Tavily durch"),
    Tool(name="wiki", func=wiki_search, description="Frage einen Begriff in Wikipedia nach")
]

In [ ]:
# Modell und ChatPrompt-Template definieren
model_name = "gpt-4o-mini"
temperature = 0.0

llm = ChatOpenAI(model=model_name, temperature=temperature)

# Prompt als Template definieren
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent mit Zugriff auf Tools."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# 🔧 Agenten-Logik
agent_runnable = create_tool_calling_agent(llm, custom_tools, prompt)

# Agenten erstellen
custom_agent = AgentExecutor(
    agent=agent_runnable,
    tools=custom_tools,
    verbose=True,
)

In [ ]:
# 🔧 Test
input = """
Erstelle eine Datei 'notiz.txt' mit dem Inhalt 'Agenten können autonom agieren. 🤖',
lies sie dann wieder,
suche nach 'Python Programmierung' im Web.
Was steht zu Taylor Swift auf Wikipedia.
"""

response = custom_agent.invoke({"input": input})

In [ ]:
mprint("## 🛠️ Hands-On Agent")
mprint("---")
mprint("**Input:**")
mprint(response['input'])
mprint("**Output**:")
mprint(response['output'])

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Kalkulation
</font></p>

Gegeben ist eine Datei, die eine Reihe von Gleichungen enthält.
Der Dateiname ist GenAI/02 data/gleichungen.txt

**Gleichung:**    
41748459 - 87226336    
92995162 * 46769739    
61530438 * 56074589    
95329602 + 45418854    
412907 + 3731910    
...

Verwenden Sie einen LangChain-Agenten mit einem Tool, um jede dieser Gleichungen zu berechnen, und erstellen Sie eine Datei ähnlich dieser:

**Ergebnisse:**  
41748459 - 87226336 = 45477877   
92995162 * 46769739 = 4349359455002718   
61530438 * 56074589 = 3450294021839982   
95329602 + 45418854 = 140748456   
412907 + 3731910 = 4144817   
... ...

